Dog-Breed-Identifier.zip 파일을 먼저 구글 드라이브에 업로드 하세요.

In [ ]:
!pip install tensorflow keras numpy pandas pillow scikit-learn


In [ ]:
pip install tensorflow keras numpy pandas


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# 압축 해제할 zip 파일 경로 (드라이브 경로를 지정합니다)
zip_file_path = '/content/drive/MyDrive/Dog-Breed-Identifier.zip'  # 파일 경로 수정

# 압축을 해제할 디렉토리 설정 (여기서는 'extracted' 디렉토리로 설정)
extract_dir = '/content'
os.makedirs(extract_dir, exist_ok=True)

# zip 파일 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f'압축 해제 완료: {extract_dir}')


압축 해제 완료: /content


In [ ]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, ReLU
from keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from PIL import Image

# 변수 설정
num_breeds = 60  # 사용할 견종 수
image_size = 224  # 이미지 크기
batch_size = 64  # 배치 사이즈
label_encoder = LabelEncoder()  # 레이블 인코더 초기화

# Google Drive 경로 설정
drive_path = '/content/Dog-Breed-Identifier/'  # 데이터 경로
labels_df = pd.read_csv(drive_path + "labels.csv")  # 라벨 데이터 읽기
train_folder = drive_path + 'train/'  # 학습 이미지 폴더 경로
test_folder = drive_path + 'test/'  # 테스트 이미지 폴더 경로

# 사용할 견종 선택
breed_list = list(labels_df['breed'].value_counts().keys())  # 견종 리스트
selected_breeds = sorted(breed_list, reverse=True)[:num_breeds * 2 + 1:2]  # 선택된 견종
labels_df = labels_df.query('breed in @selected_breeds')  # 라벨 데이터에서 선택된 견종만 필터링
labels_df['image_file'] = labels_df['id'].apply(lambda x: x + ".jpg")  # 이미지 파일명 추가

# 이미지 데이터를 numpy 배열로 변환
train_images = np.zeros((len(labels_df), image_size, image_size, 3), dtype='float32')

for i, image_id in enumerate(labels_df['image_file']):
    image = Image.open(train_folder + image_id)  # 이미지 열기
    image = image.resize((image_size, image_size))  # 이미지 크기 조정
    image_array = preprocess_input(np.expand_dims(np.array(image).astype(np.float32), axis=0))  # 이미지 전처리
    train_images[i] = image_array  # 배열에 이미지 추가

# 레이블 인코딩
train_labels = label_encoder.fit_transform(labels_df["breed"].values)

# 데이터 분할 (훈련, 검증 데이터)
x_train, x_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# 이미지 데이터 증강
data_gen_train = ImageDataGenerator(rotation_range=45,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.25,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

train_gen = data_gen_train.flow(x_train, y_train, batch_size=batch_size)  # 훈련 데이터 제너레이터

data_gen_val = ImageDataGenerator()  # 검증 데이터 제너레이터

val_gen = data_gen_val.flow(x_val, y_val, batch_size=batch_size)  # 검증 데이터 생성

# ResNet50V2 모델 불러오기
resnet_model = ResNet50V2(input_shape=[image_size, image_size, 3], weights='imagenet', include_top=False)
for layer in resnet_model.layers:
    layer.trainable = False  # 모든 레이어를 학습 불가능하게 설정

# 모델 구성
x = resnet_model.output
x = BatchNormalization()(x)  # 배치 정규화
x = GlobalAveragePooling2D()(x)  # 글로벌 평균 풀링
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.5)(x)  # 드롭아웃 추가

# 출력층
output = Dense(num_breeds, activation='softmax')(x)  # 소프트맥스 활성화 함수 사용

# 모델 생성
model = Model(inputs=resnet_model.input, outputs=output)

# 학습 설정
num_epochs = 20  # 에포크 수
learning_rate = 1e-4  # 학습률 (Adam optimizer는 보통 더 낮은 학습률을 사용)

optimizer = Adam(learning_rate=learning_rate)  # Adam 옵티마이저 사용
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=["accuracy"])  # 모델 컴파일

# 콜백 설정
checkpoint = ModelCheckpoint(filepath=drive_path + "model/best_model.keras", monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)

# 모델 학습
model.fit(train_gen,
          steps_per_epoch=x_train.shape[0] // batch_size,
          epochs=num_epochs,
          validation_data=val_gen,
          validation_steps=x_val.shape[0] // batch_size,
          callbacks=[checkpoint, early_stopping, reduce_lr])

# 최종 모델 저장
model.save(drive_path + "model/model.keras")


Epoch 1/20
64/64 [==============================] - ETA: 0s - loss: 3.7806 - accuracy: 0.1401
Epoch 1: val_loss improved from inf to 2.70991, saving model to /content/Dog-Breed-Identifier/model/best_model.keras
64/64 [==============================] - 49s 710ms/step - loss: 3.7806 - accuracy: 0.1401 - val_loss: 2.7099 - val_accuracy: 0.5527 - lr: 1.0000e-04
Epoch 2/20
64/64 [==============================] - ETA: 0s - loss: 2.2702 - accuracy: 0.4274
Epoch 2: val_loss improved from 2.70991 to 1.79402, saving model to /content/Dog-Breed-Identifier/model/best_model.keras
64/64 [==============================] - 45s 706ms/step - loss: 2.2702 - accuracy: 0.4274 - val_loss: 1.7940 - val_accuracy: 0.7041 - lr: 1.0000e-04
Epoch 3/20
64/64 [==============================] - ETA: 0s - loss: 1.7171 - accuracy: 0.5478
Epoch 3: val_loss improved from 1.79402 to 1.25964, saving model to /content/Dog-Breed-Identifier/model/best_model.keras
64/64 [==============================] - 45s 698ms/step - los

In [ ]:
import numpy as np
from keras.models import load_model
from keras.applications.resnet_v2 import preprocess_input
from PIL import Image
from google.colab import files
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# 사용할 강아지 품종 수 설정
num_dog_breeds = 60

# LabelEncoder 로드
labels_df = pd.read_csv('/content/Dog-Breed-Identifier/labels.csv')  # 라벨 CSV 파일 로드
breed_list = list(labels_df['breed'].value_counts().keys())
selected_breeds = sorted(breed_list, reverse=True)[:num_dog_breeds * 2 + 1:2]
filtered_labels_df = labels_df.query('breed in @selected_breeds')
label_encoder = LabelEncoder()
label_encoder.fit(filtered_labels_df["breed"].values)

# 모델 로드
dog_breed_model = load_model('/content/Dog-Breed-Identifier/model/model.keras')

# 이미지 업로드
uploaded_files = files.upload()

for file_name in uploaded_files.keys():
    # 이미지 로드 및 전처리
    image = Image.open(file_name)
    image = image.resize((224, 224))
    image_array = np.array(image).astype(np.float32)
    preprocessed_image = preprocess_input(np.expand_dims(image_array, axis=0))
    print(preprocessed_image.shape)

    # 예측
    prediction = dog_breed_model.predict(preprocessed_image)
    predicted_label = np.argmax(prediction, axis=1)  # 예측 결과에서 가장 높은 확률을 가진 라벨 선택
    predicted_breed = label_encoder.inverse_transform(predicted_label)  # 라벨을 실제 품종 이름으로 변환

    # 예측 결과 출력
    print(f"Predicted Breed for {file_name} is: {predicted_breed[0]}")

Saving MknyLnPac51lx9ZzAccsUKZVapDxjOkD-9qHCu-g5jburDdhn9VOg2oPE-75u5XQ_1wxWDrf1G8hQ7DLJFs-3A.webp to MknyLnPac51lx9ZzAccsUKZVapDxjOkD-9qHCu-g5jburDdhn9VOg2oPE-75u5XQ_1wxWDrf1G8hQ7DLJFs-3A.webp
(1, 224, 224, 3)
1/1 [==============================] - 1s 867ms/step
Predicted Breed for MknyLnPac51lx9ZzAccsUKZVapDxjOkD-9qHCu-g5jburDdhn9VOg2oPE-75u5XQ_1wxWDrf1G8hQ7DLJFs-3A.webp is: golden_retriever
